In [1]:
import at
from importlib.resources import files, as_file
import numpy as np

In [2]:
from at.future import ElementVariable, RefptsVariable
from at import VariableBase, AttributeVariable, ItemVariable, CustomVariable, EvaluationVariable
from at import LocalOpticsObservable, ObservableList

# Variables

Variables are **references** to any scalar quantity. Predefined classes are available
for accessing any scalar attribute of an element, or any item of an array attribute.

Any other quantity may be accessed by either subclassing the {py:class}`~.variables.VariableBase`
abstract base class, or by using a {py:class}`~.variables.CustomVariable`.

## {py:class}`~.lattice_variables.ElementVariable`

An {py:class}`~.lattice_variables.ElementVariable` refers to a single attribute (or item of an array attribute) of one or several {py:class}`.Element` objects.

We now create a variable pointing to the length of a QF1 magnet:

In [3]:
qf1 = at.Quadrupole("QF1", 0.5, 2.1)
print(qf1)

Quadrupole:
       FamName: QF1
        Length: 0.5
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.  2.1]
             K: 2.1


In [4]:
lf1 = ElementVariable(qf1, "Length", name="lf1")
print(f"{lf1}: {lf1.value}")

lf1: 0.5


and another variable pointing to the strength of the same magnet:

In [5]:
kf1 = ElementVariable(qf1, "PolynomB", index=1, name="kf1")
print(f"{kf1}: {kf1.value}")

kf1: 2.1


We can check which elements are concerned by the `kf1` variable. The element container is a set, so that no element may appear twice:

In [6]:
kf1.elements

{Quadrupole('QF1', 0.5, 2.1)}

We can now change the strength of QF1 magnets and check again:

In [7]:
kf1.set(2.5)
print(qf1)

Quadrupole:
       FamName: QF1
        Length: 0.5
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.  2.5]
             K: 2.5


We can look at the history of `kf1` values

In [8]:
kf1.history

[np.float64(2.1), 2.5]

And revert to the initial or previous values:

In [9]:
kf1.set_previous()
print(qf1)

Quadrupole:
       FamName: QF1
        Length: 0.5
    PassMethod: StrMPoleSymplectic4Pass
      MaxOrder: 1
   NumIntSteps: 10
      PolynomA: [0. 0.]
      PolynomB: [0.  2.1]
             K: 2.1


An {py:class}`~.lattice_variables.ElementVariable` is linked to Elements. It will apply wherever the element appears but it will not follow any copy of the element, neither shallow nor deep. So if we make a copy of QF1:

In [10]:
qf2 = qf1.deepcopy()
print(f"qf1: {qf1.PolynomB[1]}")
print(f"qf2: {qf2.PolynomB[1]}")

qf1: 2.1
qf2: 2.1


and modify the `kf1` variable:

In [11]:
kf1.set(2.6)

In [12]:
print(f"qf1: {qf1.PolynomB[1]}")
print(f"qf2: {qf2.PolynomB[1]}")

qf1: 2.6
qf2: 2.1


The copy of QF1 in is not affected.

One can set upper and lower bounds on a variable. Trying to set a value out of the bounds will raise a {py:obj}`ValueError`. The default is (-{py:obj}`numpy.inf`, {py:obj}`numpy.inf`).

In [13]:
lfbound = ElementVariable(qf1, "Length", bounds=(0.45, 0.55))

In [14]:
lfbound.set(0.2)

ValueError: Value 0.2 must be larger or equal to 0.45

Variables also accept a *delta* keyword argument. Its value is used as the initial step in matching, and in the {py:meth}`~.variables.VariableBase.step_up` and {py:meth}`~.variables.VariableBase.step_down` methods.

## {py:class}`.RefptsVariable`

An {py:class}`.RefptsVariable` is similar to an {py:class}`~.lattice_variables.ElementVariable` but it is not associated with an {py:class}`~.Element`
itself, but with its location in a Lattice. So it will act on any lattice with the same elements.

But it needs a *ring* keyword in its *set* and *get* methods, to identify the selected lattice.

Let's load a test ring and make a copy of it:

In [15]:
fname = "hmba.mat"
with as_file(files("machine_data") / fname) as path:
    ring = at.load_lattice(path)
newring = ring.deepcopy()

and create a {py:class}`.RefptsVariable`

In [16]:
kf2 = RefptsVariable("QF1[AE]", "PolynomB", index=1, name="kf2")

We can now use this variable on the two rings:

In [17]:
kf2.set(2.55, ring=ring)
kf2.set(2.45, ring=newring)

In [18]:
print(f"   ring:   {ring[5].PolynomB[1]}")
print(f"newring: {newring[5].PolynomB[1]}")

   ring:   2.55
newring: 2.45


## {py:class}`.AttributeVariable`

An {py:class}`.AttributeVariable` drives an attribute of an object. For example, we can drive a Lattice attribute like its
{py:attr}`~.Lattice.energy`:

In [19]:
energy_var = AttributeVariable(newring, "energy")

In [20]:
print(energy_var.value)

6000000000.0


In [21]:
energy_var.value = 6.1e9
print(newring.energy)

6100000000.0


We can look at the history of the variable

In [22]:
energy_var.history

[6000000000.0, 6100000000.0]

and go back to the initial value

In [23]:
energy_var.reset()

## {py:class}`.ItemVariable`

An {py:class}`.ItemVariable` drives an item of a directory or of a sequence.

In [24]:
obj = dict(key1=42.0, key2=[0.0, 1.0, 2.0, 3.0])
v1 = at.ItemVariable(obj, "key1")

*v1* is associated with the item "key1" of *obj* ({py:class}`dict`).

In [25]:
v2 = at.ItemVariable(obj, "key2", 1)

*v2* is associated with the 2nd item of *obj["key2"]* ({py:class}`list`)

We can look at v1, change its value and check *obj*:

In [26]:
print(v1.value)
v1.value = 0.5
print(obj)

42.0
{'key1': 0.5, 'key2': [0.0, 1.0, 2.0, 3.0]}


We can also look at v2, change its value and check *obj*:

In [27]:
print(v2.value)
v2.value = 10.0
print(obj)

1.0
{'key1': 0.5, 'key2': [0.0, 10.0, 2.0, 3.0]}


## {py:class}`.EvaluationVariable`

An {py:class}`.EvaluationVariable` drives a parameter used by the {py:meth}`.ObservableList.evaluate` method. These parameters depend on the Observables involved. For example, optics observables rely on *dp*, *orbit*, *twiss_in* … Default values may be provided when instantiating the {py:class}`.ObservableList`. An {py:class}`.EvaluationVariable` can vary these default values before the evaluation.

Here is an {py:class}`.ObservableList`:

In [28]:
twiss_in = {"alpha": np.zeros(2), "beta": np.array([9.0, 2.5])}

obs = at.ObservableList(
    [
        at.LocalOpticsObservable([0], "beta", plane="x"),
        at.LocalOpticsObservable([0], "beta", plane="y"),
    ],
    ring=ring,
    dp=0.01,
    orbit=np.zeros(6),
    twiss_in=twiss_in,
)

Create a variable controlling {math}`\delta`:

In [29]:
v1 = EvaluationVariable(obs, "dp")

Create a variable controlling {math}`p_x` and change its value:

In [30]:
v2 = EvaluationVariable(obs, "orbit", 1)
v2.value = 0.001

Create a variable controlling {math}`\beta_x`:

In [31]:
v3 = EvaluationVariable(obs, "twiss_in", "beta", 0)

We can look at their values:

In [32]:
print(v1.value)
print(v2.value)
print(v3.value)

0.01
0.001
9.0


The next evaluation of *obs* will use these values.

## Custom variables
Custom variables allow access to almost any quantity in AT. This can be achieved either by subclassing the {py:class}`~.variables.VariableBase` abstract base class, or by using a {py:class}`~.variables.CustomVariable`.

We will take 2 examples:

1. A variable accessing the {py:obj}`DPStep <.DConstant>` AT parameter used in chromaticity computations. It does not look like a very
   useful variable, it's for demonstration purpose,
2. A variable accessing the {py:attr}`~.Lattice.energy` of a given lattice. This can be done more easily by using an
   {py:class}`.AttributeVariable`, but it's also for demonstration.

### Using the {py:class}`.CustomVariable`

Using a {py:class}`~.variables.CustomVariable` makes it very easy to define simple variables: we just need
to define two functions for the "get" and "set" actions, and give them to the {py:class}`~.variables.CustomVariable` constructor.

#### Example 1

We define 2 functions for setting and getting the variable value:

In [33]:
def setvar1(value, **_):
    at.DConstant.DPStep = value


def getvar1(**_):
    return at.DConstant.DPStep

In [34]:
dpstep_var = CustomVariable(setvar1, getvar1, bounds=(1.0e-12, 0.1))
print(dpstep_var.value)

3e-06


In [35]:
dpstep_var.value = 2.0e-4
print(at.DConstant.DPStep)

0.0002


#### Example 2

We can give to the {py:class}`~.variables.CustomVariable` constructor any positional or keyword argument
necessary for the *set* and *get* functions. Here we will send the lattice as a positional argument:

In [36]:
def setvar2(value, lattice, **_):
    lattice.energy = value

def getvar2(lattice, **_):
    return lattice.energy

energy_var = CustomVariable(setvar2, getvar2, newring)

Here, the *newring* positional argument given to the variable constructor is available as a positional argument
in both the *set* and *get* functions.

In [37]:
print(energy_var.value)

6000000000.0


In [38]:
energy_var.value = 6.1e9
print(newring.energy)

6100000000.0


We can look at the history of the variable

In [39]:
energy_var.history

[6000000000.0, 6100000000.0]

and go back to the initial value

In [40]:
energy_var.reset()

### By derivation of the {py:class}`.VariableBase` class

The derivation of {py:class}`~.variables.VariableBase` allows more control on the created variable by using
the class constuctor and its arguments to setup the variable.

We will write a new variable class based on {py:class}`~.variables.VariableBase` abstract base class. The main task is to implement the `_setfun` and `_getfun` abstract methods.

#### Example 1

In [41]:
class DPStepVariable(VariableBase):

    def _setfun(self, value, ring=None):
        at.DConstant.DPStep = value

    def _getfun(self, ring=None):
        return at.DConstant.DPStep

In [42]:
dpstep_var = DPStepVariable()
print(dpstep_var.value)

0.0002


In [43]:
dpstep_var.value = 3.0e-6
print(dpstep_var.value)

3e-06


#### Example 2

Here we will store the lattice as an instance variable in the class constructor:

In [44]:
class EnergyVariable(VariableBase):
    def __init__(self, lattice):
        # Initialise the parent class
        super().__init__(lattice)

    def _setfun(self, value, lattice, **_):
        lattice.energy = value

    def _getfun(self, lattice, **_):
        return lattice.energy

We construct the variable:

In [45]:
energy_var = EnergyVariable(newring)

Look at the initial state:

In [46]:
print(energy_var.value)

6000000000.0


In [47]:
energy_var.value = 6.1e9
print(newring.energy)

6100000000.0


In [48]:
energy_var.reset()